In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import bs4 as bs4
from urllib.parse import urlparse
import requests
from collections import Counter
import pandas as pd
import re

In [2]:
pd.set_option('display.max_row', 10)
df = pd.read_csv('../website_classification.csv')
df.tail(10)

,website_url,cleaned_website_text,Category
916,https://www.cnb.gov.sg/drug-information/drugs-...,central narcotics bureau of singapore drugs an...,Narcotics
917,https://www.cnb.gov.sg/drug-information/drugs-...,central narcotics bureau of singapore drugs an...,Narcotics
918,https://www.cnb.gov.sg/drug-information/drugs-...,central narcotics bureau of singapore drugs an...,Narcotics
919,https://www.cnb.gov.sg/drug-information/drugs-...,central narcotics bureau of singapore drugs an...,Narcotics
920,https://www.cnb.gov.sg/drug-information/drugs-...,central narcotics bureau of singapore drugs an...,Narcotics
921,https://www.cnb.gov.sg/drug-information/drugs-...,central narcotics bureau of singapore drugs an...,Narcotics
922,https://www.cnb.gov.sg/drug-information/drugs-...,central narcotics bureau of singapore drugs an...,Narcotics
923,https://www.cnb.gov.sg/drug-information/drugs-...,central narcotics bureau of singapore drugs an...,Narcotics
924,https://www.cnb.gov.sg/drug-information/drugs-...,central narcotics bureau of singapore drugs an...,Narcotics
925,https://www.cnb.gov.sg/drug-information/drugs-...,central narcotics bureau of singapore drugs an...,Narcotics


In [32]:
computers = pd.read_csv('../Hierarchal model/Computer/computer_subcategory_classification.csv')
computers

,website_url,cleaned_website_text,Category
0,http://www.kasrl.org/jaffe.html,japanese female facial expression jaffe datase...,Computers and Technology
1,https://www.gmdhshell.com/,best predictive analytic software free academi...,Computers and Technology
2,http://en.wikipedia.org/wiki/Neural_network,neural network wikipedia neural network conten...,Computers and Technology
3,http://cadingandcoding.blogspot.com/,cading coding cading coding tuesday march auto...,Computers and Technology
4,http://www.panzercad.com/,welcome bluehost rate web host provider free c...,Computers and Technology
...,...,...,...
312,https://kryptomixer.io/,cryptomixer blender bitcoin mixer service cryp...,Cryptocurrency
313,https://www.walletexplorer.com/,walletexplorercom smart bitcoin block explorer...,Cryptocurrency
314,https://ultramixer.net/,ultramixer bitcoin mixer bitcoin tumbler best...,Cryptocurrency
315,https://nakedsecurity.sophos.com/2020/02/17/po...,police bust alleged operator of bitcoin mixing...,Cryptocurrency


In [33]:
from bs4 import BeautifulSoup
import bs4 as bs4
from urllib.parse import urlparse
import requests
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import os


class ScrapTool:   
    def visit_url(self,website_url):
        '''
        Visit URL. Download the Content. Initialize the beautifulsoup object. Call parsing methods. Return Series object.
        '''
        #Set up Selenium webdriver
        PATH = 'C:/Users/User01/Downloads/chromedriver_win32/chromedriver.exe'

        # Set the Chrome webdriver options
        chrome_options = Options()
        chrome_options.add_argument("--headless")
        # Initialize the Chrome webdriver and pass the Chrome options
        service = Service(PATH)
        driver = webdriver.Chrome(service = service, options = chrome_options)

        #Load website
        driver.get(website_url)
        
        # Check if the cookie consent button is present
        cookie_button = None
        try:
            cookie_button = WebDriverWait(driver, 15).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "cookie-consent-button")))
        except:
            pass

        # Handle cookies if the button is present
        if cookie_button:
            cookie_button.click()

        # Extract HTML content
        html_content = driver.page_source
        driver.quit()

        soup = BeautifulSoup(html_content,'lxml')
        result = {
            "website_url": website_url,
            "website_name": self.get_website_name(website_url),
            "website_text": self.get_html_title_tag(soup)+self.get_html_meta_tags(soup)+self.get_html_heading_tags(soup)+
                                    self.get_text_content(soup)
        }
        
        #Convert to Series object and return
        return pd.Series(result)
    
    def get_website_name(self,website_url):
        '''
        Example: returns "google" from "www.google.com"
        '''
        return "".join(urlparse(website_url).netloc.split(".")[-2])
    
    def get_html_title_tag(self,soup):
        '''Return the text content of <title> tag from a webpage'''
        return '. '.join(soup.title.contents)
    
    def get_html_meta_tags(self,soup):
        '''Returns the text content of <meta> tags related to keywords and description from a webpage'''
        tags = soup.find_all(lambda tag: (tag.name=="meta") & (tag.has_attr('name') & (tag.has_attr('content'))))
        content = [str(tag["content"]) for tag in tags if tag["name"] in ['keywords','description']]
        return ' '.join(content)
    
    def get_html_heading_tags(self,soup):
        '''returns the text content of heading tags. The assumption is that headings might contain relatively important text.'''
        tags = soup.find_all(["h1","h2","h3","h4","h5","h6"])
        content = [" ".join(tag.stripped_strings) for tag in tags]
        return ' '.join(content)
    
    def get_text_content(self,soup):
        '''returns the text content of the whole page with some exception to tags. See tags_to_ignore.'''
        tags_to_ignore = ['style', 'script', 'head', 'title', 'meta', '[document]',"h1","h2","h3","h4","h5","h6","noscript"]
        tags = soup.find_all(text=True)
        result = []
        for tag in tags:
            stripped_tag = tag.strip()
            if tag.parent.name not in tags_to_ignore\
                and isinstance(tag, bs4.element.Comment)==False\
                and not stripped_tag.isnumeric()\
                and len(stripped_tag)>0:
                result.append(stripped_tag)
        return ' '.join(result)

In [34]:
def cleaning_text(text):
    text = text.lower()
    text = text.replace('\n',' ')
    text = text.replace('\t',' ')
    text = text.replace('\r',' ')
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'\s+',' ',text)
    return text

In [35]:
def content_generation(website):
    try:
        scrapTool = ScrapTool()
        web = scrapTool.visit_url(website)
        text = cleaning_text(web['website_text'])
        return text
    except Exception as e:
        print('Error: ',website)
        print(e)
        return None

In [36]:
websites = [
    "https://adf.org.au/drug-facts/aspirin/",
    "https://adf.org.au/drug-facts/depressants/#wheel",
    "https://adf.org.au/drug-facts/alcohol/",
    "https://adf.org.au/drug-facts/benzodiazepines/",
    "https://adf.org.au/drug-facts/ghb/",
    "https://adf.org.au/drug-facts/kava/",
    "https://adf.org.au/drug-facts/dexamphetamine/",
    "https://adf.org.au/drug-facts/amphetamines/",
    "https://adf.org.au/drug-facts/caffeine/",
    "https://adf.org.au/drug-facts/empathogens/#wheel",
    "https://adf.org.au/drug-facts/ethylone/",
    "https://adf.org.au/drug-facts/mdma/",
    "https://adf.org.au/drug-facts/mephedrone/",
    "https://adf.org.au/drug-facts/pma-and-pmma/",
    "https://adf.org.au/drug-facts/vaping-e-cigarettes/",
    "https://adf.org.au/talking-about-drugs/medicinal-cannabis-products/",
    "https://adf.org.au/talking-about-drugs/medicinal-cannabis-products/what-is-medicinal-cannabis/",
    "https://www.nps.org.au/professionals/medicinal-cannabis-what-you-need-to-know",
    "https://www.cnb.gov.sg/drug-information/plants-that-contain-controlled-drugs",
    "https://www.cnb.gov.sg/drug-information/drugs-and-inhalants#Buprenorphine",
    "https://www.cnb.gov.sg/drug-information/drugs-and-inhalants#Cannabis",
    "https://www.cnb.gov.sg/drug-information/drugs-and-inhalants#BZP-and-TFMPP",
    "https://www.cnb.gov.sg/drug-information/drugs-and-inhalants#Cocaine",
    "https://www.cnb.gov.sg/drug-information/drugs-and-inhalants#Ecstasy",
    "https://www.cnb.gov.sg/drug-information/drugs-and-inhalants#Heroin",
    "https://www.cnb.gov.sg/drug-information/drugs-and-inhalants#Inhalants",
    "https://www.cnb.gov.sg/drug-information/drugs-and-inhalants#Ketamine",
    "https://www.cnb.gov.sg/drug-information/drugs-and-inhalants#Kratom",
    "https://www.cnb.gov.sg/drug-information/drugs-and-inhalants#Lysergide-LSD",
    "https://www.cnb.gov.sg/drug-information/drugs-and-inhalants#Mephedrone",
    "https://www.cnb.gov.sg/drug-information/drugs-and-inhalants#Methamphetamine",
    "https://www.cnb.gov.sg/drug-information/drugs-and-inhalants#nps",
    "https://www.cnb.gov.sg/drug-information/drugs-and-inhalants#Nimetazepam"
]

for website in websites:
    try:
        content = content_generation(website)
        print(content)
        content_list = [website,content,"Narcotics"]
        df.loc[len(df)] = content_list
    except Exception as e:
        print(website,": ",e)


asprin alcohol and drug foundationaspirin acetylsalicylic acid is a pharmaceutical drug used to reduce pain or inflammation it is classified as a nonsteroidal antiinflammatory drug nsaidaspirin what is aspirin what does aspirin look like other names how is aspirin used effects of aspirin side effects overdose longterm effects tolerance and dependence using aspirin with other drugs getting help effects akatalking about drugs having the conversation the power of words parenting talk about it vaping seeking help the law impact of aod on the workplace medicinal cannabis products aod and mental health in young people synthetic cannabinoids the facts the rise of online alcohol delivery reducing risk alcohol pharmaceuticals polydrug use how long do drugs stay in your system withdrawal relapse overdose and drugrelated emergencies responding to someone who is upset or aggressive coping with coronavirus covid staying safe at events supervised injecting facilities alcohol and other drugs and ment

In [37]:
df['Category'].value_counts()

Computers and Technology           303
Social Networking and Messaging    114
Business/Corporate                 107
E-Commerce                         100
News                                93
Narcotics                           89
Law and Government                  83
Adult                               21
Forums                              16
Name: Category, dtype: int64

In [38]:
df.to_csv("website_classification.csv",index=False)
# computers.to_csv("../Hierarchal model/Computer/computer_subcategory_classification.csv",index=False)